# 1.0 Introduction
Nas redes sociais é comum ver os usuários estarem sempre comentando sobre assuntos atuais e no Twitter isso não diferente. Considerando este fato, este trabalho se proprõe a estudar alguns padrões sociais por meior do auxílio da Análise de Redes. Primeiramente, definimos um tema a ser estudado. Considerando a data do início deste trabalho (14/02/2022), o tema escolhido foi a NFL, a principal liga de futebol americano do Estados Unidos, mais especificamente o Super Bowl que ocorreu no domingo (13/02/2022). 

Contextualizando o Super Bowl é jogo final do campeonato da NFL, que decide o campeão da temporada. Disputado desde 1967, é o maior evento desportivo e de maior audiência televisiva do país, assistido anualmente por milhões de pessoas nos Estados Unidos.

**Students:**
- Alexandre Alves
- Francisco de Assis
- Kaio Henrique


***Copyright:*** *Parts of the contents of this Colab Notebook, unless otherwise indicated, are Copyright 2020 Filippo Menczer, Santo Fortunato and Clayton A. Davis, [A First Course in Network Science](https://github.com/CambridgeUniversityPress/FirstCourseNetworkScience). All rights reserved.* 

***References***: getting started with the Twitter API v2 for academic research [here](https://github.com/twitterdev/getting-started-with-the-twitter-api-v2-for-academic-research). 


# 2.0 Authenticating with Twitter's API


O Twitter usa o OAuth para permitir o uso apps de terceiro, dessa forma não é ncessário do uso de login do Twitter. 

Mas o que é a API do Twitter? Basicamente, é por ela que um desenvolvedor pode recuperar dados públicos de usuários, tweets, retweets, etc. De fato, um desenvolvedor não tem acesso ilimitado a esses dados, dessa forma a API impõe um limite ao número de requisições que podem ser feitos, para não sobrecarregar a rede.

Nós faremos uso pacote
[Twython](https://twython.readthedocs.io/en/latest/usage/starting_out.html#authentication)
para nos auxuliar nas requisições e consultas dos tweets que buscamos.

In [ ]:
!pip install Twython

In [ ]:
from twython import Twython, TwythonError


## 2.1 Enter app info and get auth URL


Pare fazer a autentificação com Twitter, devemos fornecer a chave publica e a chave privada dadas pelo app do Twitter na conta de desenvolvedor. Dessa forma, copie e cole as chaves em um arquivo chamado <font color="red">keys.txt</font>. A primeira linha deve conter a chave pública. Segue abaixo um exemplo do arquivo <font color="red">keys.txt</font>.

```python
df6cf09894907b92f3ea749ef
d19c40cbb184f72055c806f107b5158d023a43eb7d8921a0d0
```

In [ ]:
# open the keys file
my_file = open("keys.txt", "r")

# read the raw data
content = my_file.read()

# split all lines by  newline character
API_KEY, API_SECRET_KEY = content.split("\n")

# close the file
my_file.close()

Ao executar a próxima célula, a saída será um URL clicável. O link irá funcionar apenas uma vez, após fazer o login na sua conta de desenvolvedor por meio do link gerado, o número de pin dado deve ser copiado e colado no próximo passo. 

In [ ]:
twitter = Twython(API_KEY, API_SECRET_KEY)

authentication_tokens = twitter.get_authentication_tokens()
print(authentication_tokens['auth_url'])

https://api.twitter.com/oauth/authenticate?oauth_token=GVT4jAAAAAABZKTtAAABfv15g_4


## 2.2 Authorize app using verifier PIN


Para continuar no uso da API do Twiiter vamos colar o número do PIN na variável <font color="red">VERIFIER</font>. O número de PIN tem pouco tempo de validade e será diferente sempre que os passos anteriores forem executados. Os `authentication_tokens` são tokens temporários que são associados ao número de PIN gerado.

In [ ]:
# Replace the verifier with the pin number obtained with your web browser in the previous step
VERIFIER = '2887399'

twitter = Twython(API_KEY, API_SECRET_KEY,
                  authentication_tokens['oauth_token'],
                  authentication_tokens['oauth_token_secret'])

authorized_tokens = twitter.get_authorized_tokens(VERIFIER)


## 2.3 Use authorized tokens


Com posse do par de tokens podemos fazer chamadas a API do Twitter por meio de um objeto do Twython.

In [ ]:
twitter = Twython(API_KEY, API_SECRET_KEY,
                  authorized_tokens['oauth_token'],
                  authorized_tokens['oauth_token_secret'])

twitter.verify_credentials()

Se a célula anterior executar sem erro e printar um dicionário correpondendo a um [Usuário do Twitter](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object), então está tudo certo para continuar. Tokens de autenticação são como um par usuário/senha e devem ser protegidos da mesma forma.


# 3.0 Twitter retweet network


Para o construção da nossa network, vamos utilizar de umas das interações mais básicas presentes no Twitter os retweets, mas para isso primeiro iremos explicar sobre como um retweet funciona. Um tweet é um publicação feita por um usuário em que o conteúdo dessa publicação é compartilhado com os seguidores desse usuário. Dessa forma, quando o usuário faz o retweet de um tweet já existente, o tweet original é transmitido em broadcast para os seguidores do usuário que fez o retweet, ou seja, é uma republicação que inclui informações do dono do tweet.

Um objeto do tipo tweet retornado pela API é um retweet quando inclui um atributo `'retweeted_status'`. Neste trabalho, vamos buscar por tweets que contenham a hashtag `#NFL` e criar uma network de retweets.

## 3.1 Create DiGraph


Na construção da network cada nó representa um usuário do Twitter e um aresta direcionada é adicionada quando um usuário faz um retweet, a direção da aresta vai do usuário que fez o tweet para o usuário que fez o retweet. Como um usuário pode retweetar um mesmo usuário mais de uma vez, iremos adicionar peso às arestas com número de retweets representado pelo peso.

In [ ]:
import datetime
import pandas as pd
import networkx as nx
import time

D = nx.DiGraph()
dict_ = {'id_retweet': [], 'retweeted_screen_name': [], 'retweeted_location': [], 'retweeter_screen_name': [], 'retweeter_location' : []}

# Getting today's date
datestamp = datetime.datetime.now().strftime("%Y-%m-%d")


In [ ]:
import itertools

NUM_TWEETS_TO_FETCH = 15000

cursor = twitter.cursor(twitter.search, q='#NFL', count=100, result_type='mixed')
search_tweets = []
#search_tweets = list(itertools.islice(cursor, NUM_TWEETS_TO_FETCH))
#len(search_tweets)

A celula abaixo é executada duas vezes, logo recuperamos ao todo 60.000 tweets que dentres estes, apenas 28.585 são retweets. 

In [ ]:
for ii in range(2): # loop para coletar dados a cada 16 minutos
    search_tweets.extend(list(itertools.islice(cursor, NUM_TWEETS_TO_FETCH)))
    time.sleep(16 * 60)

print(len(search_tweets))

60000


In [ ]:
retweets = []
for tweet in search_tweets:
    if 'retweeted_status' in tweet:
        retweets.append(tweet)
print("filter ", len(retweets))

filter  28585


Aqui iteramos sobre a lista de retweets e adicionamos a arestas com peso 1 caso ainda não exista, caso contrário acrescentamos 1 ao peso da aresta existente.

In [ ]:
for retweet in retweets:
    retweeted_status = retweet['retweeted_status']

    retweeted_sn = retweeted_status['user']['screen_name']
    retweetedL = retweeted_status['user']['location']
    retweeter_sn = retweet['user']['screen_name']
    retweeterL = retweet['user']['location']

    dict_['id_retweet'].append(retweet["id"])
    dict_['retweeted_screen_name'].append(retweeted_sn)
    dict_['retweeted_location'].append(retweetedL)
    dict_['retweeter_screen_name'].append(retweeter_sn)
    dict_['retweeter_location'].append(retweeterL)
  
    # Edge direction: retweeted_sn -> retweeter_sn
    if D.has_edge(retweeted_sn, retweeter_sn):
        D.edges[retweeted_sn, retweeter_sn]['weight'] += 1
    else:
        D.add_edge(retweeted_sn, retweeter_sn, weight=1)
    

No passo seguinte salvamos em um CSV um DataFrame com as seguinte colunas:
- id_retweet (id do retweet)
- retweeted_screen_name `(O usuário dono do tweet)`
- retweeted_location `(Localição definida pelo usuário)`
- retweeter_screen_name `(O usuário que fez o retweet)`
- retweeter_location `(Localição definida pelo usuário)`

In [ ]:
# criando dataframe de retweets com #NFL 
df = pd.DataFrame(dict_)

df.head()

,id_retweet,retweeted_screen_name,retweeted_location,retweeter_screen_name,retweeter_location
0,1493582820241031170,MackTightRadio,Worldwide,SapphireSteamy,🙏🏽 LEVITATED 🙏🏽
1,1493582749919236100,therealBeede,"Orlando, FL",_0wayz,
2,1493582683070472199,DavidMTodd,,jaduke77,"Pittsburgh, PA"
3,1493582582298341382,crypto_prince2,"Las Vegas, NV",AkmazRasim,
4,1493582582264516610,NFLBrasil,Brasil,IgorBSilva81,"Valinhos, Brasil"


In [ ]:
# Salvando dados em csv
df.to_csv("NFL-"+datestamp+".csv")

Em um arquivo .graphml salvamos a network construida no NetworkX.

In [ ]:
# Salvando network em graphml
nx.write_graphml(D, "NFL-"+datestamp+".graphml")

In [ ]:
# Verificando a quantidade de requests restantes
twitter.get_application_rate_limit_status()['resources']['search']

{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1644944119}}

## 6.2 Analyze graph



### 6.2.1 Most retweeted user


Como as arestas da nossa rede são em direção do fluxo de informação, logo o grau de saída nos retorna o número de usuários que retweetaram um determinado usuários. Portanto, podemos verficar o usuário que foi mais retweetado no nosso tema (NFL), dessa forma verificamos a pessoa ou a página com mais voz dentro do esporte.

In [ ]:
max(D.nodes, key=D.out_degree)

'Brother_nfts'

Também é interessante verificar o contexto e mais informações sobre o Top 5 mais retweeetados.

In [ ]:
from operator import itemgetter

sorted(D.out_degree(), key=itemgetter(1), reverse=True)[:5]

[('Brother_nfts', 1543),
 ('Endzone_Brasil', 1173),
 ('1218Sports', 519),
 ('jollenelevid', 488),
 ('OddsCheckerUS', 482)]

In [ ]:
D.out_degree()

OutDegreeView({'CarlaZambelli38': 105, 'CarlosBolzan2': 0, 'RFransceschi': 0, 'JosCarl78233530': 0, 'lssposito': 0, 'ercio_santoss': 0, 'BrunoCr62058963': 0, 'cleide_ita': 0, 'JFH84343564': 0, 'rblondt': 0, 'LeiHigor': 0, 'ovasco71': 0, 'Eduardoegg2': 0, 'sissa155': 0, 'Claudinho_oa': 0, 'HackAlberto': 0, 'danilovsouza1': 0, 'EldriEldri': 0, 'MachadoPrudente': 0, 'belluccis': 0, 'MarcosDiaslogan': 0, 'PedroFe33848000': 0, 'NovaFriburgoRJ': 0, 'Lou_novak': 0, 'ReinaldoLuizCa2': 0, 'maceno_sueli': 0, 'Docilda1': 0, 'soniaalmeidafe': 0, 'JooBati47318744': 0, 'regisrpop': 0, 'PrRobsonAlencar': 0, 'Marcos08905454': 0, 'Washing41753473': 0, 'JuNascimentoGyn': 0, 'EzioDiasdoNasc1': 0, 'RobsonWiller3': 0, 'BrasilPtriaAma3': 0, 'almagnolima': 0, 'soniaTangari': 0, 'UiraitanReis': 0, 'Paiakkan': 0, 'angoneto': 0, 'Salvado89779435': 0, 'dudu_santana05': 0, 'Raimund39337518': 0, 'ClaudomiroSil18': 0, 'Renan1debora': 0, 'Arlindo71123942': 0, 'NellsBhor': 0, 'RicardoLipex': 0, 'WOLF_Lorn': 0, 'Richa

No trecho seguinte adicionamos a tag `weight` para que o peso das arestas sejam contadas diferente do que fizemos no código anterior e assim analizamos os **Nós** que possuem mais arestas *saindo* para outros **Nós**, ou seja, retornamos as 5 contas ou páginas que são mais interagidas com relação a ***hashtag*** pela comunidade do Twitter.

In [ ]:
sorted(D.out_degree(weight='weight'), key=itemgetter(1), reverse=True)[:5]

[('Endzone_Brasil', 3305),
 ('Brother_nfts', 1673),
 ('1218Sports', 1013),
 ('nflextra', 843),
 ('jollenelevid', 765)]

Note que diferente do primeiro Top 5 que geramos, esse possui a conta `Endzone_Brasil` como maior soma de peso de arestas, isso ocorre quando uma mesma conta Retweeta mais de uma vez um mesmo Tweet, mas na maioria das vezes encontramos sempre uma grande semelhança entre os dois Top 5, que normalmente são idênticos.


### 6.2.2 Anomaly detection


Como em redes sociais a visibilidade de Tweet pode aumentar baseado na quantidade de Retweets, é possível identificar contas destinadas apenas para compartilhar o conteúdo de outras contas de maneira manual o automática, no caso, bots. pode identificar tais contas pelas seguinte linha de código.

In [ ]:
sorted(D.in_degree(weight='weight'), key=itemgetter(1), reverse=True)[:5]

[('theffrobot', 153),
 ('topfanscorner', 143),
 ('nflttbr', 111),
 ('touchdownbot', 95),
 ('iglen31', 47)]

### 6.2.3 Connectivity


Podemos também descobrir se os dados que estamos lidando são todos ligados entre si, ou seja, que trata-se de uma grande conversação. O código seguinte nos mostra que não estamos com uma estrutura com tal propriedade.

In [ ]:
nx.is_weakly_connected(D)

False

In [ ]:
nx.number_weakly_connected_components(D)

2453